<a href="https://colab.research.google.com/github/wind-yu/lhy-ML/blob/main/lhy_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

from tqdm.auto import tqdm

In [6]:
# Google Drive
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

!unzip -q food-11.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
To: /content/food-11.zip
100% 963M/963M [00:16<00:00, 58.6MB/s]


In [7]:
#It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    #transforms.Resize((128, 128)),
    transforms.RandomResizedCrop((128, 128)),
    transforms.RandomChoice([
        transforms.AutoAugment(),
        transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
        transforms.AutoAugment(transforms.AutoAugmentPolicy.SVHN),
    ]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2), scale=(0.7, 1.3)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

# We don't need augmentation in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [8]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

# Contruct datasets.
# The argument "loader" tells how torchvision reads the data.
def loader_fn(x):
    return Image.open(x)

dataset_path = "food-11/"
train_set = DatasetFolder(dataset_path + "training/labeled", loader=loader_fn, extensions="jpg", transform=train_tfm)
#print(dataset_path + "training/labeled")
valid_set = DatasetFolder(dataset_path + "validation", loader=loader_fn, extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder(dataset_path + "training/unlabeled", loader=loader_fn, extensions="jpg", transform=train_tfm)
test_set = DatasetFolder(dataset_path + "testing", loader=loader_fn, extensions="jpg", transform=test_tfm)

# Construct data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [16]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # http://9ghsck.cc/vodplay/28876-1-1.html
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(512, 1024, 3, 1, 1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(1024*4*4, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(256, 11)
        )
        
    def forward(self, X):
        # input (X): [batch_size, 3, 128, 128]
        # output : [batch_size, 11]
        
        # Extract features by convolutional layers.
        X = self.cnn_layers(X)

        # The extracted features map must be flatten before going to fully connected layers.
        X = X.flatten(1)

        # The features are transformed by fully connected layers to obtain the final logits.
        X = self.fc_layers(X)

        return X
        

In [10]:
def get_pseudo_labels(dataset, model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)

    #Iterate over the dataset by batches.
    for batch in tqdm(dataloader):
        img, _ = batch

        # Forward the data
        # Using torch.no_grad() accelerates the forward process
        with torch.no_grad():
            logits = model(img.to(device))

        probs = softmax(logits)

        # TODO:
        # Filter the data and construct a new dataset

    # Ture off the eval mode.
    model.train()
    
    return dataset

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performence.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on you own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# The number of training epochs
n_epochs = 250

# Whether to do semi-supervised learning.
do_semi = False

for epoch in range(n_epochs):
    # TODO
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled for the training.
    if do_semi:
        # Obtain pseudo-labels for unlabeled data using trained model.
        pseudo_set = get_pseudo_labels(unlabeled_set, model)

        # Construct a new dataset and a data loader for training.
        # This is used in semi-supervised learning only.
        concat_dataset = ConcatDataset([train_set, pseudo_set]) 
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    # -------- Training ---------
    # Make sure the model is in train mode before training.
    model.train()
    
    # These are used to record information in training.
    train_loss = []
    train_accs = []
    
    # Iterate the training set by batches.
    for batch in tqdm(train_loader):
        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        # Forward the data. (Make sure data and model are on the same device).
        logits = model(imgs.to(device))

        # Calculate the corss-entropy loss.
        # We don't need to apply softmax before computing corss-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()
        
        # Compute the gradients for parameters.
        loss.backward()
        
        # Clip the gradient norms for stable training
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients
        optimizer.step()
        
        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        
        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss: .5f}, acc = {train_acc: .5f}")

    # ----------- Validation -------------
    # Make sure the model is in eval mode so that modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation
    valid_loss = []
    valid_accs = []

    # Iteration the validation set by batches.
    for batch in tqdm(valid_loader):
        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We donit need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        
        # Record the loss and accuracy.
        valid_loss.append(loss.item())

        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d}] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


  0%|          | 0/25 [00:00<?, ?it/s]

[ Train | 001/250 ] loss =  2.46295, acc =  0.10469


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 001/250] loss = 2.50931, acc = 0.09219


  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

[ Train | 085/250 ] loss =  1.73013, acc =  0.39969


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 085/250] loss = 1.56333, acc = 0.47552


  0%|          | 0/25 [00:00<?, ?it/s]